# Day 5

This builds on [day 2](./02.ipynb) but adds several new modes. 

 * New Input and Output functions
 * Variable parameter operations
 * Parameter modes
 * Immediate mode operations
 
 
We will change our operation spec to take current memory pointer and memory and then return the new memory pointer. Return a negative number for an exit code.



 

In [ ]:
# First we need to support parameter mode
def parse_operation(oper):
    value = "{:05}".format(oper)
    assert int(value) == oper
    assert len(value) == 5
    
    a = int(value[0])
    b = int(value[1])
    c = int(value[2])
    op = int(value[3:])
    
    return dict(op=op, a=a, b=b, c=c)
    
assert parse_operation(1002) == dict(op=2, a=0, b=1, c=0)


In [ ]:
def test_oper(oper, memory, ptr=0):
    op = parse_operation(memory[ptr])
    return oper(ptr, op['a'], op['b'], op['c'], memory), memory

In [ ]:
def add(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    c = memory[ptr+3]
    
    if mc == 0:
        a = memory[a]
    
    if mb == 0:
        b = memory[b]
    
    # Output is always in position mode
    memory[c] = a + b
    
    return ptr+4

assert test_oper(add, [1001,4,3,4,0]) == (4, [1001, 4, 3, 4, 3])

In [ ]:
def multiply(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    c = memory[ptr+3]

    if mc == 0:
        a = memory[a]
    
    if mb == 0:
        b = memory[b]
        
    # Output is always in position mode
    memory[c] = a * b
    
    return ptr+4

test_oper(multiply, [1002,4,3,4,33]) == (4, [1002,4,3,4,99])

First, you'll need to add two new instructions:

* Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
* Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.

In [ ]:
def user_input(ptr, ma, mb, mc, memory):
    value = int(input())
    memory[memory[ptr+1]] = value
    return ptr+2


In [ ]:
def user_output(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    if mc == 0:
        a = memory[a]

    print("OUTPUT:", a)
    return ptr+2  

In [ ]:
def stop(ptr, ma, mb, mc, memory):
    return -1

test_oper(stop, [1002,4,3,4,99]) == (-1, [1002,4,3,4,99])

In [ ]:
operations = {
    1: add,
    2: multiply,
    3: user_input,
    4: user_output,
    99: stop
    }

def execute(ptr, memory):
    op = parse_operation(memory[ptr])
#     print(ptr, op, memory[:20])
    operation = operations[op['op']]
    return operation(ptr, op['a'], op['b'], op['c'], memory)
    
def run(memory):
    pointer = 0
    memory = list(memory)
    while True:
        pointer = execute(pointer, memory)
        if pointer < 0:
            break
        
    return pointer, memory
        
       
run([1002,4,3,4,33])
assert run([1002,4,3,4,33]) == (-1, [1002, 4, 3, 4, 99])


In [ ]:
with open("05-input.txt", "rt") as FILE:
    data = FILE.read()
    data = data.split(",")
    data = [int(d) for d in data]
data[:8]

In [ ]:
output = run(data)

## Part 2

Your computer is only missing a few opcodes:

* Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

* Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

* Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

* Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.


In [ ]:
def jump_if_true(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    if mc == 0:
        a = memory[a]
    if mb == 0:
        b = memory[b]
    
    if a == 0:
        return ptr+3
    else:
        return b
    
def jump_if_false(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    if mc == 0:
        a = memory[a]
    if mb == 0:
        b = memory[b]
    
    if a == 0:
        return b
    else:
        return ptr+3

def less_than(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    c = memory[ptr+3]
    if mc == 0:
        a = memory[a]
    if mb == 0:
        b = memory[b]

    if a < b:
        memory[c] = 1
    else:
        memory[c] = 0

    return ptr+4
    
def equals(ptr, ma, mb, mc, memory):
    a = memory[ptr+1]
    b = memory[ptr+2]
    c = memory[ptr+3]
    if mc == 0:
        a = memory[a]
    if mb == 0:
        b = memory[b]

    if a == b:
        memory[c] = 1
    else:
        memory[c] = 0
        
    return ptr+4


In [ ]:
operations = {
    1: add,
    2: multiply,
    3: user_input,
    4: user_output,
    5: jump_if_true,
    6: jump_if_false,
    7: less_than,
    8: equals,
    99: stop
}

In [ ]:
# Taking the input 8 - this should return 1, otherwise 0
run([3,9,8,9,10,9,4,9,99,-1,8])

In [ ]:
# 3,9,7,9,10,9,4,9,99,-1,8 - Using position mode, consider whether the input is less than 8; 
# output 1 (if it is) or 0 (if it is not).

run([3,9,7,9,10,9,4,9,99,-1,8])

In [ ]:
# 3,3,1108,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is equal to 8; 
# output 1 (if it is) or 0 (if it is not).

run([3,3,1108,-1,8,3,4,3,99])

In [ ]:
# 3,3,1107,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is less than 8; 
# output 1 (if it is) or 0 (if it is not).

run([3,3,1107,-1,8,3,4,3,99])

In [ ]:
# Here are some jump tests that take an input, then output 0 if the input was zero or 1 if the input was non-zero:

# 3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9 (using position mode)
# 3,3,1105,-1,9,1101,0,0,12,4,12,99,1 (using immediate mode)

run([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9])

In [ ]:
run([3,3,1105,-1,9,1101,0,0,12,4,12,99,1])

In [ ]:
# The above example program uses an input instruction to ask for a single number. 
# The program will then output 999 if the input value is below 8, output 1000 if 
# the input value is equal to 8, or output 1001 if the input value is greater than 8.

a = run([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,
         21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99])

In [ ]:
output = run(data)